In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip uninstall transformers -Y
    %pip install transformers

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/mres/model_harvesting


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -Y
Mounted at /content/gdrive
/content/gdrive/MyDrive/mres/model_harvesting


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
from IPython.display import clear_output

In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def free_mem(vars):
    for v in vars: del v
    gc.collect()
    t.cuda.empty_cache()

In [4]:
models = {
    "llama2": "meta-llama/Llama-2-7b-chat-hf",
    "llama3": "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistral": "mistralai/Mistral-7B-v0.1"
}

model_name = "llama3"
model = AutoModelForCausalLM.from_pretrained(models[model_name], torch_dtype=t.float16).to(device); model.eval()
tokenizer = AutoTokenizer.from_pretrained(models[model_name])

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tk_A = tokenizer.encode("A", add_special_tokens=False)[0]
tk_B = tokenizer.encode("B", add_special_tokens=False)[0]

aspects = ["coherence", "consistency", "fluency", "relevance"]

### newsroom

In [ ]:
prompts = pd.read_json(f"prompts_short/newsroom-theirs-compare-llama3.jsonl", orient="records", lines=True)
for aspect in aspects:
    p_s1, p_s2 = [], []
    for i in trange(len(prompts), desc=aspect):
        prompt = prompts.at[prompts.index[i], f"prompt-{aspect}"]
        tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
        with t.no_grad(): out = model(tks)
        logits = out["logits"][:, -1, [tk_A, tk_B]]
        P = F.softmax(logits, dim=-1)
        p_s1.append(P[:, 0].item())
        p_s2.append(P[:, 1].item())
        free_mem([tks, logits, P])
    prompts["p_s1"] = p_s1
    prompts["p_s2"] = p_s2
    prompts.to_json(f"{model_name}/logits_short/{aspect}.jsonl", orient="records", lines=True)

coherence:   0%|          | 0/1260 [00:00<?, ?it/s]

consistency:   0%|          | 0/1260 [00:00<?, ?it/s]

fluency:   0%|          | 0/1260 [00:00<?, ?it/s]

relevance:   0%|          | 0/1260 [00:00<?, ?it/s]

### summeval

In [ ]:
prompts = pd.read_json(f"prompts_short/summeval-theirs-compare-llama3.jsonl", orient="records", lines=True)
for aspect in aspects:
    p_s1, p_s2 = [], []
    for i in trange(len(prompts), desc=aspect):
        prompt = prompts.at[prompts.index[i], f"prompt-{aspect}"]
        tks = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False).to(device)
        with t.no_grad(): out = model(tks)
        logits = out["logits"][:, -1, [tk_A, tk_B]]
        P = F.softmax(logits, dim=-1)
        p_s1.append(P[:, 0].item())
        p_s2.append(P[:, 1].item())
        free_mem([tks, logits, P])
    prompts["p_s1"] = p_s1
    prompts["p_s2"] = p_s2
    prompts.to_json(f"{model_name}/logits_short/summeval-{aspect}.jsonl", orient="records", lines=True)

coherence:   0%|          | 0/12000 [00:00<?, ?it/s]

consistency:   0%|          | 0/12000 [00:00<?, ?it/s]

fluency:   0%|          | 0/12000 [00:00<?, ?it/s]

relevance:   0%|          | 0/12000 [00:00<?, ?it/s]